In [ ]:
import pandas as pd
import numpy as np

# Load the merged data
merged_csv_path = '../data/processed/train_merged.csv'
df = pd.read_csv(merged_csv_path, low_memory=False, parse_dates=['Date'])

print('Data loaded successfully.')
df.head()

### 1. Tarih Verisinden Özellik Türetme

In [ ]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

print('Date features created.')
df[['Date', 'Year', 'Month', 'Day', 'DayOfWeek', 'WeekOfYear']].head()

### 2. Rekabet İle İlgili Özellikler

In [ ]:
# Eksik değerleri doldurma (örneğin 0 veya medyan ile)
df['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
df['CompetitionOpenSinceYear'].fillna(0, inplace=True)

# Rakibin ne kadar süredir açık olduğunu ay olarak hesapla
df['CompetitionOpen'] = (df['Year'] - df['CompetitionOpenSinceYear']) * 12 + \
                        (df['Month'] - df['CompetitionOpenSinceMonth'])
# Negatif değerleri 0 yap (eğer yarışma gelecek bir tarihte açılıyorsa)
df['CompetitionOpen'] = df['CompetitionOpen'].apply(lambda x: max(x, 0))

print('Competition features created.')
df[['Date', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'CompetitionOpen']].head()

### 3. Promo2 İle İlgili Özellikler

In [ ]:
# Eksik değerleri doldurma
df['Promo2SinceWeek'].fillna(0, inplace=True)
df['Promo2SinceYear'].fillna(0, inplace=True)
df['PromoInterval'].fillna('', inplace=True)

def is_promo2_active(row):
    if row['Promo2'] == 0:
        return 0
    
    # Promo2 başlangıç tarihini ve şu anki tarihi karşılaştır
    promo2_start_year = int(row['Promo2SinceYear'])
    promo2_start_week = int(row['Promo2SinceWeek'])
    current_year = row['Year']
    current_week = row['WeekOfYear']

    if current_year < promo2_start_year:
        return 0
    if current_year == promo2_start_year and current_week < promo2_start_week:
        return 0

    # Promo ayında olup olmadığını kontrol et
    month_str = row['Date'].strftime('%b') # Ay kısaltması (Jan, Feb, ...)
    if month_str in row['PromoInterval']:
        return 1
    else:
        return 0

df['IsPromo2'] = df.apply(is_promo2_active, axis=1)

print('Promo2 features created.')
df[df['Promo2'] == 1][['Date', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'IsPromo2']].head(10)